In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay, roc_curve, roc_auc_score, mean_squared_error, r2_score,, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor,export_text
from sklearn.preprocessing import OneHotEncoder

In [ ]:
data =  pd.read_csv('MasterDataset_WithWeatherAndEvent.csv')

data.head()
len(data)

In [ ]:
data = data.drop(data.columns[[0,1,2]], axis=1)
data = data.drop(["date", "time"], axis=1)
data.head()
len(data)

In [ ]:
print(f"Total Number of rows: {len(data)}")
print(f"Total number of null values: {data.isnull().sum().sum()}")

print(100*(data.isnull().sum())/len(data))

In [ ]:
data['holiday'].fillna("Not a holiday", inplace=True)
data = data.dropna(how='any',axis=0)

In [ ]:
print(len(data))
data.head()

In [ ]:
data[['Start date', 'End date']] = data[['Start date', 'End date']].apply(pd.to_datetime)
data[['Start station', 'Start station number', 'End station','End station number', 'Member type', 'holiday']] = data[['Start station', 'Start station number', 'End station','End station number', 'Member type', 'holiday']].astype('category')
data[["Duration", "temperature_2m", "relativehumidity_2m", "precipitation", "windspeed_10m"]] = data[["Duration", "temperature_2m", "relativehumidity_2m", "precipitation", "windspeed_10m"]].astype('float')
data["isHoliday"] = data["isHoliday"].astype("boolean")

In [ ]:
data[["year", "month", "m_day", "hour"]] = data["Start date"].apply(lambda x: x.timetuple()[0:4]).tolist()
data["w_day"] = data["Start date"].apply(lambda x: x.weekday())

In [ ]:
len(data)

In [ ]:
cols = ["Duration"] 
Q1 = data[cols].quantile(0.25)
Q3 = data[cols].quantile(0.75)
IQR = Q3 - Q1
data = data[~((data[cols] < (Q1 - 1.5 * IQR)) |(data[cols] > (Q3 + 1.5 * IQR))).any(axis=1)]
after_outlier = len(data)

In [ ]:
print(len(data))

In [ ]:
data["Member type"] = data["Member type"].str.lower()
data = data[data["Start date"].dt.year < 2020]

In [ ]:
print(len(data[data['Duration'] <= 0]))


In [ ]:
print(f"Total Number of rows: {len(data)}")
print(f"Total number of null values: {data.isnull().sum().sum()}")

print(100*(data.isnull().sum())/len(data))

In [ ]:
selected_columns = ['Duration', 'isHoliday', 'month', 'm_day', 'hour', 'w_day', 'temperature_2m', 'relativehumidity_2m', 'precipitation', 'windspeed_10m']
Xlog = data[selected_columns]
ylog = data['Member type'].map({'casual': 0, 'member': 1})


Xlog_train, Xlog_test, ylog_train, ylog_test = train_test_split(Xlog, ylog, test_size=0.3, random_state=65)

model_log = LogisticRegression(solver='liblinear')
fit = model_log.fit(Xlog_train, ylog_train)

predictions_log = model_log.predict(Xlog_test)

print("Accuracy:", accuracy_score(ylog_test, predictions_log))



In [ ]:
Xlog = sm.add_constant(Xlog)  
model_log1 = sm.Logit(ylog, Xlog)
result = model_log1.fit()

p_values = result.pvalues
print(p_values)

In [ ]:
coefficients_log = model_log.coef_[0]  
intercept_log = model_log.intercept_[0]

precision = precision_score(ylog_test, predictions_log)

print("Accuracy:", accuracy_score(ylog_test, predictions_log))
print(f"Precision: {precision:.4f}")
print(f"Sensitivity:", recall_score(ylog_test, predictions_log))
print()
sorted(range(len(coefficients_log)), key=lambda k: abs(coefficients_log[k]), reverse=True)

for index in sorted_indices:
    feature = Xlog.columns[index]
    coef = coefficients_log[index]
    print(f"{feature}: {coef}")

print()
print("Intercept:", intercept_log)

In [ ]:
coefficients_log = model_log.coef_[0]
intercept_log = model_log.intercept_[0]

precision = precision_score(ylog_test, predictions_log)

print("\nSorted Coefficients (Descending Order):")
sorted_indices = sorted(range(len(coefficients_log)), key=lambda k: abs(coefficients_log[k]), reverse=True)

for index in sorted_indices:
    feature = Xlog.columns[index]
    coef = coefficients_log[index]
    print(f"{feature}: {coef}")

print("Intercept:", intercept_log)


In [ ]:
conf_matrix = confusion_matrix(ylog_test, predictions_log)

disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=model_log.classes_)
disp.plot()
plt.title('Confusion Matrix')
plt.show()

In [ ]:
TN, FP, FN, TP = conf_matrix.ravel()

accuracy = accuracy_score(ylog_test, predictions_log)
precision = precision_score(ylog_test, predictions_log, average='weighted')
sensitivity = recall_score(ylog_test, predictions_log, average='weighted')

print("\nMetrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Sensitivity: {sensitivity:.4f}")

In [ ]:
y_probs = model_log.predict_proba(Xlog_test)[:, 1]
y_test_binary = (ylog_test == 'member').astype(int)

fpr, tpr, thresholds = roc_curve(y_test_binary, y_probs)

roc_auc = roc_auc_score(y_test_binary, y_probs)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC Curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--', lw=2, label='Random')
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.show()

In [ ]:
features = ['month', 'm_day', 'hour', 'w_day', 'temperature_2m', 'relativehumidity_2m', 'precipitation', 'windspeed_10m', 'isHoliday']
X_lin = data[features]
y_lin = data['Duration']

Xlin_train, Xlin_test, ylin_train, ylin_test = train_test_split(X_lin, y_lin, test_size=0.3, random_state=65)

model_lin = LinearRegression()
model_lin.fit(Xlin_train, ylin_train)

predictions_lin = model_lin.predict(Xlin_test)

mse_lin = mean_squared_error(ylin_test, predictions_lin)
r2_lin = r2_score(ylin_test, predictions_lin)

print("Model Evaluation:")
print(f"Mean Squared Error (MSE): {mse_lin:.4f}")
print(f"R-squared (R2): {r2_lin:.4f}")

plt.scatter(ylin_test, predictions_lin)
plt.xlabel("Actual Duration")
plt.ylabel("Predicted Duration")
plt.title("Actual vs. Predicted Duration")
plt.show()

In [ ]:
coefficients = model1.coef_
coefficients

In [ ]:
#Checking assumptions
# Homoscedasticity
residuals1 = y1_test - predictions1
plt.scatter(predictions1, residuals1)
plt.axhline(y=0, color='red', linestyle='--')
plt.xlabel("Predicted Durations")
plt.ylabel("Residuals")
plt.title("Homoscedasticity Check")
plt.show()


In [ ]:
# Normality of Residuals
sns.histplot(residuals1, kde=True)
plt.xlabel("Residuals")
plt.ylabel("Frequency")
plt.title("Normality of Residuals Check")
plt.show()

In [ ]:
#model performance is very poor and the linearity assumptions dont hold so we will do some feature selection

In [ ]:
correlation_matrix = data.corr()

plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Matrix")
plt.show()

In [ ]:
#will remove relative humidity because it has high correlation with hour

In [ ]:
data.shape()

In [ ]:
features2 = ['month', 'm_day', 'hour', 'w_day', 'temperature_2m', 'precipitation', 'windspeed_10m', 'isHoliday']
X2 = data[features]
y1 = data['Duration']

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y1, test_size=0.3, random_state=65)

model2 = LinearRegression()
model2.fit(X2_train, y2_train)

predictions2 = model2.predict(X2_test)

mse_2 = mean_squared_error(y2_test, predictions2)
r2_2= r2_score(y2_test, predictions2)

print("Model Evaluation:")
print(f"Mean Squared Error (MSE): {mse_2:.4f}")
print(f"R-squared (R2): {r2_2:.4f}")

plt.scatter(y2_test, predictions2)
plt.xlabel("Actual Duration")
plt.ylabel("Predicted Duration")
plt.title("Actual vs. Predicted Duration")
plt.show()

In [ ]:
y1 = data['Duration']
X1 = data[['month', 'm_day', 'hour', 'w_day']]
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.3, random_state=42)

model1 = LinearRegression()

model1.fit(X1_train, y1_train)

y1_pred = model1.predict(X1_test)

mse = mean_squared_error(y1_test, y1_pred)
print(f'Mean Squared Error: {mse}')

r2_linear = r2_score(y1_test, y1_pred)

print(f'R-squared: {r2_linear}')

In [ ]:
X_dt = data[features_dt]
y_dt = data['Duration']

cat_features = ['Member type', 'holiday']
X_dt_cat = X_dt[cat_features]

X_dt_non_cat = X_dt.drop(cat_features, axis=1)

print("Length of X_dt:", len(X_dt))
print("Length of X_dt_cat:", len(X_dt_cat))
print("Length of X_dt_non_cat:", len(X_dt_non_cat))

In [ ]:
encoder = OneHotEncoder(drop='first', sparse=False)
X_dt_encoded = pd.DataFrame(encoder.fit_transform(X_dt_cat))
encoded_columns = list(encoder.get_feature_names_out(cat_features))
X_dt_encoded.columns = encoded_columns

print("Length of X_dt_encoded:", len(X_dt_encoded))

In [ ]:
X_dt_non_cat.reset_index(drop=True, inplace=True)
X_dt_encoded.reset_index(drop=True, inplace=True)

print("Length of X_dt_non_cat (after reset index):", len(X_dt_non_cat))
print("Length of X_dt_encoded (after reset index):", len(X_dt_encoded))

In [ ]:
features_dt= ['Start station number', 'End station number',  'Member type', 'temperature_2m', 'relativehumidity_2m', 'precipitation', 'windspeed_10m', 'holiday', 'isHoliday', 'month', 'm_day', 'hour', 'w_day']
X_dt = data[features_dt]
y_dt = data['Duration']

cat_features = ['Member type', 'holiday']
X_dt_cat = X_dt[cat_features]

X_dt_non_cat = X_dt.drop(cat_features, axis=1)


encoder = OneHotEncoder(drop='first', sparse=False)
X_dt_encoded = pd.DataFrame(encoder.fit_transform(X_dt_cat))
encoded_columns = list(encoder.get_feature_names_out(cat_features))
X_dt_encoded.columns = encoded_columns


X_dt_non_cat.reset_index(drop=True, inplace=True)
X_dt_encoded.reset_index(drop=True, inplace=True)

X_dt = pd.concat([X_dt_non_cat, X_dt_encoded], axis=1)

Xdt_train, Xdt_test, ydt_train, ydt_test = train_test_split(X_dt, y_dt, test_size=0.3, random_state=42)

model_dt = DecisionTreeRegressor(random_state=42)
model_dt.fit(Xdt_train, ydt_train)

ydt_pred = model_dt.predict(Xdt_test)

mse_dt = mean_squared_error(ydt_test, ydt_pred)
mae_dt = mean_absolute_error(ydt_test, ydt_pred)

y_dt_pred = model_dt.predict(X_dt)

r2 = r2_score(y_dt, y_dt_pred)

print(f'R-squared: {r2}')

print(f'Mean Squared Error: {mse_dt}')
print(f'Mean Absolute Error: {mae_dt}')

In [ ]:
feature_importances = tree_clf.feature_importances_

num_features = 5
top_features_indices = np.argsort(feature_importances)[-num_features:]
top_feature_importances = feature_importances[top_features_indices]
top_feature_names = Xdt_train.columns[top_features_indices]

plt.barh(top_feature_names, top_feature_importances)
plt.xlabel('Feature Importance')
plt.title('Top Features by Importance')
plt.show()

In [ ]:
feature_importances = model_dt.feature_importances_

feature_importance_df = pd.DataFrame({'Feature': X_dt.columns, 'Importance': feature_importances})

feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

print("Top Features:")
print(feature_importance_df)

plt.figure(figsize=(10, 6))
plt.bar(range(len(feature_importances)), feature_importances, align='center')
plt.xticks(range(len(feature_importances)), X_dt.columns, rotation=90)
plt.xlabel('Feature')
plt.ylabel('Importance')
plt.title('Feature Importances')
plt.show()


In [ ]:

top_features = ['End station number', 'Start station number', 'Member type_Member',
                'temperature_2m', 'windspeed_10m', 'relativehumidity_2m',
                'm_day', 'hour', 'w_day', 'month', 'year', 'precipitation',  'holiday_Not a holiday', 'isHoliday']

X_dt_top_features = X_dt[top_features]

Xdt_train_top, Xdt_test_top, ydt_train_top, ydt_test_top = train_test_split(X_dt_top_features, y_dt, test_size=0.3, random_state=42)

model_dt_top_features = DecisionTreeRegressor(random_state=42)
model_dt_top_features.fit(Xdt_train_top, ydt_train_top)

ydt_pred_top_features = model_dt_top_features.predict(Xdt_test_top)

mse_top_features = mean_squared_error(ydt_test_top, ydt_pred_top_features)
mae_top_features = mean_absolute_error(ydt_test_top, ydt_pred_top_features)

print(f'Mean Squared Error (Top Features): {mse_top_features}')
print(f'Mean Absolute Error (Top Features): {mae_top_features}')

In [ ]:
r2_top_features = r2_score(ydt_test_top, ydt_pred_top_features)
print(f'R-squared (Top Features): {r2_top_features}')
